# Tucker decomposition

In [3]:
import numpy as np
from tensorly.decomposition import tucker
from tensorly import tucker_to_tensor

# Generate a random tensor
np.random.seed(0)
tensor = np.random.rand(4, 5, 6)

# Define the rank for the Tucker decomposition
rank = (3, 3, 3)

# Perform Tucker decomposition
core, factors = tucker(tensor, rank=rank)

# Reconstruct the tensor from the decomposed components
reconstructed_tensor = tucker_to_tensor((core, factors))

# Print the original tensor, core tensor, factors, and reconstructed tensor
print("Original Tensor:")
print(tensor)
print("\nCore Tensor:")
print(core)
print("\nFactors:")
for i, factor in enumerate(factors):
    print(f"Factor {i}:")
    print(factor)
print("\nReconstructed Tensor:")
print(reconstructed_tensor)

# Calculate and print the reconstruction error
reconstruction_error = np.linalg.norm(tensor - reconstructed_tensor)
print(f"\nReconstruction Error: {reconstruction_error}")

Original Tensor:
[[[0.5488135  0.71518937 0.60276338 0.54488318 0.4236548  0.64589411]
  [0.43758721 0.891773   0.96366276 0.38344152 0.79172504 0.52889492]
  [0.56804456 0.92559664 0.07103606 0.0871293  0.0202184  0.83261985]
  [0.77815675 0.87001215 0.97861834 0.79915856 0.46147936 0.78052918]
  [0.11827443 0.63992102 0.14335329 0.94466892 0.52184832 0.41466194]]

 [[0.26455561 0.77423369 0.45615033 0.56843395 0.0187898  0.6176355 ]
  [0.61209572 0.616934   0.94374808 0.6818203  0.3595079  0.43703195]
  [0.6976312  0.06022547 0.66676672 0.67063787 0.21038256 0.1289263 ]
  [0.31542835 0.36371077 0.57019677 0.43860151 0.98837384 0.10204481]
  [0.20887676 0.16130952 0.65310833 0.2532916  0.46631077 0.24442559]]

 [[0.15896958 0.11037514 0.65632959 0.13818295 0.19658236 0.36872517]
  [0.82099323 0.09710128 0.83794491 0.09609841 0.97645947 0.4686512 ]
  [0.97676109 0.60484552 0.73926358 0.03918779 0.28280696 0.12019656]
  [0.2961402  0.11872772 0.31798318 0.41426299 0.0641475  0.69247212]

In [6]:
import numpy as np
from tensorly.decomposition import tucker
from tensorly import tucker_to_tensor

# Generate a random tensor
np.random.seed(0)
tensor = np.random.rand(4, 5, 6)

# Define the rank for the Tucker decomposition
rank = (3, 3, 3)

# Perform Tucker decomposition
core, factors = tucker(tensor, rank=(4, 4))

# Reconstruct the tensor from the decomposed components
reconstructed_tensor = tucker_to_tensor((core, factors))

# Print the original tensor, core tensor, factors, and reconstructed tensor
print("Original Tensor:")
print(tensor)
print("\nCore Tensor:")
print(core)
print("\nFactors:")
for i, factor in enumerate(factors):
    print(f"Factor {i}:")
    print(factor)
print("\nReconstructed Tensor:")
print(reconstructed_tensor)

# Calculate and print the reconstruction error
reconstruction_error = np.linalg.norm(tensor - reconstructed_tensor)
print(f"\nReconstruction Error: {reconstruction_error}")

IndexError: tuple index out of range

# Constrained Tucker

In [7]:
import random
import numpy as np
import optuna
from functools import partial
from _src import WarcraftObjective, TuckerSampler


def build_constraint_warcraft(map_shape: tuple[int, int]) -> np.ndarray:
    # Directions dictionary
    directions_dict = {
        "oo": np.array([0, 0]),
        "ab": np.array([1, 1]),
        "ac": np.array([0, 2]),
        "ad": np.array([1, 1]),
        "bc": np.array([1, 1]),
        "bd": np.array([2, 0]),
        "cd": np.array([1, 1]),
    }

    directions_list = list(directions_dict.keys())

    # Map parameters
    map_length = map_shape[0] * map_shape[1]
    ideal_gain = (map_shape[0] + map_shape[1] - 1) * 2

    # Initialize constraints as NumPy arrays
    tensor_constraint_1 = np.zeros((len(directions_list),) * map_length)
    tensor_constraint_2 = np.zeros((len(directions_list),) * map_length)
    tensor_constraint_3 = np.zeros((len(directions_list),) * map_length)

    # Constraint 1: (0, 0) != "oo", "ab"
    for direction in directions_list:
        if direction not in ["oo", "ab"]:
            # tensor_constraint_1[..., directions_list.index(direction)] = 1
            tensor_constraint_1[directions_list.index(direction), ...] = 1

    # Constraint 2: (map_shape[0] - 1, map_shape[1] - 1) != "oo", "cd"
    for direction in directions_list:
        if direction not in ["oo", "cd"]:
            # tensor_constraint_2[directions_list.index(direction), ...] = 1
            tensor_constraint_2[..., directions_list.index(direction)] = 1

    # Constraint 3: len[path] == map_shape[0] * map_shape[1]
    for index, _ in np.ndenumerate(tensor_constraint_3):
        gain = np.sum([directions_dict[directions_list[idx]].sum() for idx in index])
        if gain == ideal_gain:
            tensor_constraint_3[index] = 1

    # Combine constraints with logical AND
    tensor_constraint = np.logical_and(
        tensor_constraint_1,
        np.logical_and(tensor_constraint_2, tensor_constraint_3)
    )

    return tensor_constraint


def objective(trial, map_shape=None, objective_function=None):
    directions = ["oo", "ab", "ac", "ad", "bc", "bd", "cd"]
    x = np.empty(map_shape, dtype=object)
    for i in range(map_shape[0]):
        for j in range(map_shape[1]):
            x[i, j] = trial.suggest_categorical(f"x_{i}_{j}", directions)
    return objective_function(x)


def get_map(map_option: int):
    if map_option == 1:
        map_targeted = np.array([[1, 4], [2, 1]])
    elif map_option == 2:
        map_targeted = np.array([[1, 4, 1], [2, 1, 1]])
    elif map_option == 3:
        map_targeted = np.array([[1, 4, 1], [2, 1, 3], [5, 2, 1]])
    else:
        raise ValueError(f"Invalid map option: {map_option}")
    return map_targeted / map_targeted.sum()


def run_bo():
    settings = {
        "name": "test_study",
        "seed": 1,
        "map_option": 2,
        "iter_bo": 100,
        "unique_sampling": False,
        "decomp_num": 10,
        "cp_settings": {
            "rank": 3,
            "als_iterations": 100,
            "mask_ratio": 0.9,
            "include_observed_points": False,
        },
        "acqf_settings": {
            "acquisition_function": "ei",
            "trade_off_param": 3.0,
            "batch_size": 1,
            "maximize": False,
        },
        "n_startup_trials": 20,
    }

    random.seed(settings['seed'])

    map_targeted = get_map(settings["map_option"])
    map_shape = map_targeted.shape

    tensor_constraint = build_constraint_warcraft(map_shape)

    objective_function = WarcraftObjective(map_targeted, tensor_constraint=tensor_constraint)
    objective_with_args = partial(objective, map_shape=map_shape, objective_function=objective_function)

    sampler = TuckerSampler(
        tucker_rank_base=settings["cp_settings"]["rank"],
        als_iter_num=settings["cp_settings"]["als_iterations"],
        mask_ratio=settings["cp_settings"]["mask_ratio"],
        acquisition_function=settings["acqf_settings"]["acquisition_function"],
        trade_off_param=settings["acqf_settings"]["trade_off_param"],
        seed=settings["seed"],
        unique_sampling=settings["unique_sampling"],
        decomp_iter_num=settings["decomp_num"],
        n_startup_trials=settings["n_startup_trials"],
        include_observed_points=settings["cp_settings"].get("include_observed_points", False),
        tensor_constraint=tensor_constraint,
    )

    direction = "maximize" if settings["acqf_settings"]["maximize"] else "minimize"

    study = optuna.create_study(
        study_name=settings["name"],
        sampler=sampler,
        direction=direction,
    )

    study.optimize(objective_with_args, n_trials=settings["iter_bo"])

    best_x = np.empty(map_shape, dtype=object)
    for i in range(map_shape[0]):
        for j in range(map_shape[1]):
            best_x[i, j] = study.best_params[f"x_{i}_{j}"]

    optuna.visualization.plot_optimization_history(study)



if __name__ == "__main__":
    run_bo()

[I 2024-12-07 17:44:05,704] A new study created in memory with name: test_study
[I 2024-12-07 17:44:05,706] Trial 0 finished with value: 10.0 and parameters: {'x_0_0': 'ab', 'x_0_1': 'cd', 'x_0_2': 'cd', 'x_1_0': 'oo', 'x_1_1': 'bc', 'x_1_2': 'bd'}. Best is trial 0 with value: 10.0.
[I 2024-12-07 17:44:05,707] Trial 1 finished with value: 10.0 and parameters: {'x_0_0': 'bc', 'x_0_1': 'ac', 'x_0_2': 'ad', 'x_1_0': 'bd', 'x_1_1': 'cd', 'x_1_2': 'ad'}. Best is trial 0 with value: 10.0.
[I 2024-12-07 17:44:05,708] Trial 2 finished with value: 0.9444444444444444 and parameters: {'x_0_0': 'ad', 'x_0_1': 'oo', 'x_0_2': 'cd', 'x_1_0': 'bc', 'x_1_1': 'oo', 'x_1_2': 'bd'}. Best is trial 2 with value: 0.9444444444444444.
[I 2024-12-07 17:44:05,709] Trial 3 finished with value: 10.0 and parameters: {'x_0_0': 'bd', 'x_0_1': 'bd', 'x_0_2': 'oo', 'x_1_0': 'bc', 'x_1_1': 'bc', 'x_1_2': 'ac'}. Best is trial 2 with value: 0.9444444444444444.
[I 2024-12-07 17:44:05,709] Trial 4 finished with value: 10.0 

KeyboardInterrupt: 

In [31]:
tensor.shape

(7, 7, 7, 7)

In [32]:
7**4

2401

In [33]:
tensor.sum()

np.int64(300)

In [42]:
arr = np.random.randint(0, 2, (4, 3, 2))
arr

array([[[0, 1],
        [0, 1],
        [1, 1]],

       [[0, 0],
        [1, 0],
        [1, 0]],

       [[0, 0],
        [1, 1],
        [0, 1]],

       [[0, 0],
        [1, 1],
        [1, 1]]])

In [46]:
arr[(0, 0, 0)], arr[(-1, -1, -1)], arr[(0, -1, -1)]

(np.int64(0), np.int64(1), np.int64(1))

In [47]:
arr[(0, -1, -1)] = 2
arr

array([[[0, 1],
        [0, 1],
        [1, 2]],

       [[0, 0],
        [1, 0],
        [1, 0]],

       [[0, 0],
        [1, 1],
        [0, 1]],

       [[0, 0],
        [1, 1],
        [1, 1]]])

In [ ]:
arr = np.array([
    ["aa", "ab", "ac"],
    ["ba", "bb", "bc"],
    
])

In [ ]:
import numpy as np

# 元のリスト
elements = ["a", "b", "c", "d"]

# 2文字の組み合わせを作成し、2次元配列に整形
arr = np.array([[f"{x}{y}" for y in elements] for x in elements])

print(arr)

[['aa' 'ab' 'ac' 'ad']
 ['ba' 'bb' 'bc' 'bd']
 ['ca' 'cb' 'cc' 'cd']
 ['da' 'db' 'dc' 'dd']]


In [53]:
arr[0, ...]

array(['aa', 'ab', 'ac', 'ad'], dtype='<U2')

In [48]:
import numpy as np

# 元のリスト
elements = ["a", "b", "c", "d"]

# 3つの要素の組み合わせを3階テンソルとして生成
tensor = np.array([[[f"{x}{y}{z}" for z in elements] 
                     for y in elements] 
                     for x in elements])

print(tensor)


[[['aaa' 'aab' 'aac' 'aad']
  ['aba' 'abb' 'abc' 'abd']
  ['aca' 'acb' 'acc' 'acd']
  ['ada' 'adb' 'adc' 'add']]

 [['baa' 'bab' 'bac' 'bad']
  ['bba' 'bbb' 'bbc' 'bbd']
  ['bca' 'bcb' 'bcc' 'bcd']
  ['bda' 'bdb' 'bdc' 'bdd']]

 [['caa' 'cab' 'cac' 'cad']
  ['cba' 'cbb' 'cbc' 'cbd']
  ['cca' 'ccb' 'ccc' 'ccd']
  ['cda' 'cdb' 'cdc' 'cdd']]

 [['daa' 'dab' 'dac' 'dad']
  ['dba' 'dbb' 'dbc' 'dbd']
  ['dca' 'dcb' 'dcc' 'dcd']
  ['dda' 'ddb' 'ddc' 'ddd']]]


In [50]:
tensor[(..., 0)]

array([['aaa', 'aba', 'aca', 'ada'],
       ['baa', 'bba', 'bca', 'bda'],
       ['caa', 'cba', 'cca', 'cda'],
       ['daa', 'dba', 'dca', 'dda']], dtype='<U3')

In [54]:
tensor[0, ...]

array([['aaa', 'aab', 'aac', 'aad'],
       ['aba', 'abb', 'abc', 'abd'],
       ['aca', 'acb', 'acc', 'acd'],
       ['ada', 'adb', 'adc', 'add']], dtype='<U3')

In [4]:
import numpy as np

In [5]:
def build_constraint_warcraft(map_shape: tuple[int, int]) -> np.ndarray:
    # Directions dictionary
    directions_dict = {
        "oo": np.array([0, 0]),
        "ab": np.array([1, 1]),
        "ac": np.array([0, 2]),
        "ad": np.array([1, 1]),
        "bc": np.array([1, 1]),
        "bd": np.array([2, 0]),
        "cd": np.array([1, 1]),
    }

    directions_list = list(directions_dict.keys())

    # Map parameters
    map_length = map_shape[0] * map_shape[1]
    ideal_gain = (map_shape[0] + map_shape[1] - 1) * 2

    # Initialize constraints as NumPy arrays
    tensor_constraint_1 = np.zeros((len(directions_list),) * map_length)
    tensor_constraint_2 = np.zeros((len(directions_list),) * map_length)
    tensor_constraint_3 = np.zeros((len(directions_list),) * map_length)

    # Constraint 1: (0, 0) != "oo", "ab"
    for direction in directions_list:
        if direction not in ["oo", "ab"]:
            tensor_constraint_1[directions_list.index(direction), ...] = 1

    # Constraint 2: (map_shape[0] - 1, map_shape[1] - 1) != "oo", "cd"
    for direction in directions_list:
        if direction not in ["oo", "cd"]:
            tensor_constraint_2[..., directions_list.index(direction)] = 1

    # Constraint 3: len[path] == map_shape[0] * map_shape[1]
    for index, _ in np.ndenumerate(tensor_constraint_3):
        gain = np.sum([directions_dict[directions_list[idx]].sum() for idx in index])
        if gain == ideal_gain:
            tensor_constraint_3[index] = 1

    # Combine constraints with logical AND
    tensor_constraint = np.logical_and(
        tensor_constraint_1,
        np.logical_and(tensor_constraint_2, tensor_constraint_3)
    )

    return tensor_constraint

In [6]:
tensor = build_constraint_warcraft((1, 2))

In [8]:
~tensor

array([[ True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True],
       [ True, False, False, False, False, False,  True],
       [ True, False, False, False, False, False,  True],
       [ True, False, False, False, False, False,  True],
       [ True, False, False, False, False, False,  True],
       [ True, False, False, False, False, False,  True]])